In [ ]:
import sys

sys.path.append('../3_analyze')
from containers import Dataset
from utils import plot_docking

get_ipython().magic(u'matplotlib inline')

In [ ]:
gpcrs = ['B1AR','B2AR','A2AR']
kinases = ['BRAF','CDK2','CHK1','JAK2','PLK1']
nuclear = ['AR','ERA','GCR','VITD']

proteins = ['BRAF']

In [ ]:
all_data = Dataset(proteins)

ligands = {p:prot.lm.pdb for p,prot in all_data.proteins.items()}
st = {p:prot.lm.grids for p,prot in all_data.proteins.items()}
all_data.load(ligands, st, load_fp=False, load_mcss=False)

In [ ]:
n_list = [1,5,25,100,300]
show_only_st_that_all_docked = False

for prot, p in sorted(all_data.proteins.items()):
    for st, s in sorted(p.docking.items()):
        all_rmsds = [ [] for n in n_list ]

        rmsds = s.glide_perf(n_list=n_list, ligands=ligands[prot])
        if show_only_st_that_all_docked and None in rmsds[0]: continue
        for i, n in enumerate(n_list):
            all_rmsds[i].extend(rmsds[i])
        
        plot_docking(all_rmsds, ['top-{}'.format(n) for n in n_list], '{}, {}'.format(prot, st))